### Imports and setup

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("~/pings/ping_data/output_2024-01-20.csv")

In [ ]:
# Capture a new column that has no strings, NaN's where we see strings (errors, or time outs)
df['float_latency'] = pd.to_numeric(df['Latency (in ms)'], errors='coerce')

In [ ]:
df.head(5)

### Analysis

In [ ]:
# Count nan's in the latency column
drops = df['float_latency'].isnull().sum()

# What's the percentage of ping drops?
drop_rate = drops/len(df)

print(f'There were {drops} drops out of {len(df)} pings, or {drop_rate*100:.3g}%')



In [ ]:
# Let's drop the nan's and do some base analytics on the successful pings.
pings_list = df['float_latency'].apply(pd.to_numeric, errors='coerce')

print(pings_list.describe())